In [47]:
#### from PIL import Image
import numpy as np
#img=Image.open('ORL/s1/res8.png')
#img.show()
import re
from matplotlib import pyplot as plt
import numpy
M1=32
M2=6
M3=10
def read_pgm(filename, byteorder='>'):
    """Return image data from a raw PGM file as numpy array.

    Format specification: http://netpbm.sourceforge.net/doc/pgm.html

    """
    with open(filename, 'rb') as f:
        buffer = f.read()
    try:
        header, width, height, maxval = re.search(
            b"(^P5\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n]\s)*)", buffer).groups()
    except AttributeError:
        raise ValueError("Not a raw PGM file: '%s'" % filename)
    return numpy.frombuffer(buffer,
                            dtype='u1' if int(maxval) < 256 else byteorder+'u2',
                            count=int(width)*int(height),
                            offset=len(header)
                            ).reshape((int(height), int(width)))


if __name__ == "__main__":
    database=[]
    X=[]
    for i in range(32):
        temp=[]
        for j in range(6):
            image = read_pgm("../images/ORL/s"+str(i+1)+"/"+str(j+1)+".pgm", byteorder='<')
            temp.append(image)
            X.append(image.ravel())
        database.append(temp)
    database=np.array(database)
    X=np.array(X).T
    #X = (X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
    X = X.T
    Xb=np.sum(X,axis=0)
    Xb=Xb/X.shape[0]
    Xib=np.subtract(X,Xb)
    L=np.matmul(Xib,Xib.T)   #np.cov(Xib)
    print(L.shape)
    print(X.shape,X.shape[0])
    print(Xib.shape)
    #image = read_pgm("ORL/s1/1.pgm", byteorder='<')
    #print(type(image))
    #plt.imshow(database[31][1], pyplot.cm.gray)

(192, 192)
(192, 10304) 192
(192, 10304)


Method: we get false positives and negatives when we use the same eigen vectors for recognizing new person which should be rejected. We used the median of deviation and a parameter gamma for tuning the best possible value for rejecting the false negatives and positives. We will be accepting the values which are in range of (0,gamma*median of deviations )of image from the other images eigen vectors.The d greater than that range will be rejected. 


In [59]:
import sys
from numpy import linalg as LA
def eigcof(U,k,Xb,zp):
    Zpb=np.subtract(zp,Xb)
    vkb=U[:,:k]
    #print(vkb.shape,Zpb.shape)
    return np.matmul(np.transpose(vkb),Zpb)
def diff(alphas,u,K,Xb,image):
    a,b,c=alphas.shape
    mins=[sys.maxsize,0,0]
    vec=eigcof(u,K,Xb,image.ravel())
    #print(vec)
    for i in range(a):
        for j in range(b):
            #print(vec.shape,alphas[i,j].shape)
            res=np.subtract(vec,alphas[i,j])
            val=LA.norm(res)
            #print(vec,alphas[i,j],val)
            if(val<mins[0]):
                #print(vec,alphas[i,j],val,i,j)
                mins=[val,i,j]
    return mins[1]+1#[mins[1]+1,mins[2]+1]
def diff_extended(alphas,u,K,Xb,image,gamma=0.15):
    a,b,c=alphas.shape
    mins=[sys.maxsize,0,0]
    vec=eigcof(u,K,Xb,image.ravel())
    #print(vec)
    avg = []
    for i in range(a):
        #sum = 0
        for j in range(b):
            #print(vec.shape,alphas[i,j].shape)
            res=np.subtract(vec,alphas[i,j])
            val=LA.norm(res)
            #print(vec,alphas[i,j],val)
            #sum = sum+val
            if(val<mins[0]):
                #print(vec,alphas[i,j],val,i,j)
                mins=[val,i,j]
            avg.append(val)
    med = np.median(avg)
    if mins[0]< gamma*med:
        return mins[1]+1#[mins[1]+1,mins[2]+1]
    else:
        return 0 #not identified
def onestep(Alphas,u,K,Xb):
    Tot=0
    Pos=0
    for i in range(32):
        for j in range(6,10):
            image = read_pgm("../images/ORL/s"+str(i+1)+"/"+str(j+1)+".pgm", byteorder='<')
            chk=diff(Alphas,u,K,Xb,image.ravel())
            if(chk==(i+1)):
                Pos+=1
            Tot+=1
    return Pos/Tot
def Aplcal(u,K,Xb):
    Alphas=[]
    for i in range(32):
        temp=[]
        for j in range(6):
            image = read_pgm("../images/ORL/s"+str(i+1)+"/"+str(j+1)+".pgm", byteorder='<')
            temp.append(eigcof(u,K,Xb,image.ravel()))
        Alphas.append(temp)
    Alphas=np.array(Alphas)
    return Alphas

In [60]:
from numpy import linalg as LA
w, v = LA.eig(L)
print(Xib.shape,v.shape,w.shape)
U=np.matmul(Xib.T,v)
U = (U-U.min(axis=0))/(U.max(axis=0)-U.min(axis=0))
#U=U / np.linalg.norm(U)
print(U.shape)
u=U

(192, 10304) (192, 192) (192,)
(10304, 192)


In [61]:
image = read_pgm("../images/ORL/s"+str(1)+"/"+str(7)+".pgm", byteorder='<')
print(eigcof(u,3,Xb,image.ravel()).shape)
K=20
Alphas=[]
for i in range(32):
        temp=[]
        for j in range(6):
            image = read_pgm("../images/ORL/s"+str(i+1)+"/"+str(j+1)+".pgm", byteorder='<')
            temp.append(eigcof(u,K,Xb,image.ravel()))
        Alphas.append(temp)
Alphas=np.array(Alphas)

(3,)


In [62]:
Tot=0
Pos=0
for i in range(32):
        temp=[]
        for j in range(6,10):
            image = read_pgm("../images/ORL/s"+str(i+1)+"/"+str(j+1)+".pgm", byteorder='<')
            chk=diff(Alphas,u,K,Xb,image.ravel())
            if(chk==(i+1)):
                Pos+=1
            Tot+=1
print(Pos,Tot,'over')

94 128 over


In [63]:
recog = []
Tot=0
Pos=0
for i in range(32,40):
        temp=[]
        for j in range(10):
            image = read_pgm("../images/ORL/s"+str(i+1)+"/"+str(j+1)+".pgm", byteorder='<')
            chk=diff_extended(Alphas,u,K,Xb,image.ravel())
            recog.append(chk)
            if(chk==(i+1)):
                Pos+=1
            Tot+=1
print(Pos,Tot,'over')

0 80 over


In [64]:
print(len(recog) - np.count_nonzero(recog),len(recog))

50 80


We are able to get 30/80 wrong which are false positives detected which should be no recognitions i.e. 37.5%

In [65]:
Tot=0
Pos=0
for i in range(32):
        temp=[]
        for j in range(6,10):
            image = read_pgm("../images/ORL/s"+str(i+1)+"/"+str(j+1)+".pgm", byteorder='<')
            chk=diff_extended(Alphas,u,K,Xb,image.ravel())
            if(chk==(i+1)):
                Pos+=1
            Tot+=1
print(Pos,Tot,'over')

64 128 over


We are getting 64/128 values correct i.e 50% rejected. 